In [64]:
import requests
from pprint import pprint
import config
import numpy as np

username = 'delphinus6'
game_index = 0

response = requests.get("https://na1.api.riotgames.com/lol/summoner/v3/summoners/by-name/%s?api_key=%s" % (username, config.api_key))
summoner_data = response.json()

summoner_id = summoner_data['id']
account_id = summoner_data['accountId']

# response = requests.get('https://na1.api.riotgames.com/lol/league/v3/positions/by-summoner/%s?api_key=%s' % (summoner_id, config.api_key))
# league_data = response.json()

# print('%s\'s Winrates' % username)
# print('----')
# for league in league_data:
#     wins = league['wins']
#     losses = league['losses']
#     winrate = wins / (wins + losses) * 100
#     print('%s: %.02f%% over %d games' % (league['queueType'], winrate, (wins+losses)))
# print('----')    

response = requests.get("https://na1.api.riotgames.com/lol/match/v3/matchlists/by-account/%s?api_key=%s" % (account_id, config.api_key))
all_match_data = response.json()

latest_match_id = all_match_data['matches'][game_index]['gameId']

response = requests.get("https://na1.api.riotgames.com/lol/match/v3/matches/%s?api_key=%s" % (latest_match_id, config.api_key))
latest_match_data = response.json()

num_players = len(latest_match_data['participantIdentities']) / 2
queue_id = latest_match_data['queueId']
if queue_id == 470:
    queue_type = 'RANKED_FLEX_TT'
elif queue_id == 420:
    queue_type = 'RANKED_SOLO_5x5'
elif queue_id == 440:
    queue_type = 'RANKED_FLEX_SR'
else:
    queue_type = 'QUEUE ID NOT ACCOUNTED FOR'
    
print('Queue type: %s' % queue_type)
print('Queue id: %d' % queue_id)
print('----')

team1 = []
team1_w = 0
team1_tot = 0
team2 = []
team2_w = 0
team2_tot = 0
for i, participant in enumerate(latest_match_data['participantIdentities']):
    if i == 0:
        print('Team 1:')
    elif i == num_players:
        print('\nTeam 2:')
    
    summoner_name = participant['player']['summonerName']
    summoner_id = participant['player']['summonerId']
#     print(summoner_name, summoner_id)
    
    response = requests.get('https://na1.api.riotgames.com/lol/league/v3/positions/by-summoner/%s?api_key=%s' % (summoner_id, config.api_key))
    league_data = response.json()
#     print(league_data)
#     print(queue_type)
    league = [elt for elt in league_data if elt['queueType'] == queue_type][0]
    wins = league['wins']
    losses = league['losses']
    winrate = wins / (wins + losses)
    if latest_match_data['teams'][0]['teamId'] == latest_match_data['participants'][i]['teamId']:
#     if i < num_players:
        team1.append(winrate)
        team1_w += wins
        team1_tot += (wins + losses)
    else:
        team2.append(winrate)
        team2_w += wins
        team2_tot += (wins + losses)
    print('%s: %s/%s (%.02f%%), %s %s' % (summoner_name, wins, wins + losses, winrate * 100, league['tier'], league['rank'] ))

print('----')
    
team1_sum = sum(team1)
team2_sum = sum(team2)
print('Predictions (Team 1 vs Team2):\n')
print('    Cumulative winrates: %.02f%% vs %.02f%%' % (team1_sum*100, team2_sum*100))
team1_win = latest_match_data['teams'][0]['win'] == 'Win'
# team1_predict = team1_sum > team2_sum

sum_ratio = team1_sum / team2_sum
print('    Win likelihood from sums: %.02f%% vs %.02f%%' % (1/(1+1/sum_ratio)*100, 1/(1+sum_ratio)*100))

# Sort for min
team1.sort(reverse = False)
team2.sort(reverse = False)
ratios = np.array(team1) / np.array(team2)
team1_ratio = np.mean(ratios) / np.mean(1/ratios)
print('    Win likelihood from mean min: %.02f%% vs %.02f%%' % (1/(1+1/team1_ratio)*100, 1/(1+team1_ratio)*100))

# Sort for max
team1.sort(reverse = False)
team2.sort(reverse = True)
ratios = np.array(team1) / np.array(team2)
team1_ratio = np.mean(ratios) / np.mean(1/ratios)
max_lh_1 = 1/(1+1/team1_ratio)*100
max_lh_2 = 1/(1+team1_ratio)*100
print('    Win likelihood from mean max: %.02f%% vs %.02f%%' % (max_lh_1, max_lh_2))

map_diff = 0.026
print('    After map side difference (%.02f%%): %.02f%% vs %.02f%%' % (map_diff, max_lh_1 * (1 - map_diff), max_lh_2 * (1 + map_diff)))

team1_carry = team1[-1]/team2[-1]
team2_carry = team2[0]/team1[0]
min_carry = min(team1_carry, team2_carry)
print('    Solo carry likelihood: %.02f vs %.02f' % (team1_carry/min_carry, team2_carry/min_carry))

if team1_win:
    winner = 'Team 1'
else:
    winner = 'Team 2'
print('\nActual result: %s won!' % winner) 
# print('-->', team1_win == team1_predict)
    


Queue type: RANKED_FLEX_TT
Queue id: 470
----
Team 1:
TheKillSecure: 20/41 (48.78%), GOLD IV
Falunithik : 34/71 (47.89%), GOLD V


IndexError: list index out of range

In [1]:
# real-time
import requests
from pprint import pprint
import config
import numpy as np

username = 'SylowP'

response = requests.get("https://na1.api.riotgames.com/lol/summoner/v3/summoners/by-name/%s?api_key=%s" % (username, config.api_key))
summoner_data = response.json()

summoner_id = summoner_data['id']
account_id = summoner_data['accountId']

# response = requests.get('https://na1.api.riotgames.com/lol/league/v3/positions/by-summoner/%s?api_key=%s' % (summoner_id, config.api_key))
# league_data = response.json()

# print('%s\'s Winrates' % username)
# print('----')
# for league in league_data:
#     wins = league['wins']
#     losses = league['losses']
#     winrate = wins / (wins + losses) * 100
#     print('%s: %.02f%% over %d games' % (league['queueType'], winrate, (wins+losses)))
# print('----')    

response = requests.get("https://na1.api.riotgames.com/lol/spectator/v3/active-games/by-summoner/%s?api_key=%s" % (summoner_id, config.api_key))
current_match_data = response.json()
# print(current_match_data)

num_players = len(current_match_data['participants']) / 2
queue_id = current_match_data['gameQueueConfigId']
if queue_id == 470:
    queue_type = 'RANKED_FLEX_TT'
elif queue_id == 420:
    queue_type = 'RANKED_SOLO_5x5'
elif queue_id == 440:
    queue_type = 'RANKED_FLEX_SR'
else:
    queue_type = 'QUEUE ID NOT ACCOUNTED FOR'
    
print('Queue type: %s' % queue_type)
print('Queue id: %d' % queue_id)
print('----')

team1 = []
team2 = []
for i, participant in enumerate(current_match_data['participants']):
    try:
        if i == 0:
            print('Team 1:')
        elif i == num_players:
            print('\nTeam 2:')

        summoner_name = participant['summonerName']
        summoner_id = participant['summonerId']
    #     print(summoner_name, summoner_id)

        response = requests.get('https://na1.api.riotgames.com/lol/league/v3/positions/by-summoner/%s?api_key=%s' % (summoner_id, config.api_key))
        league_data = response.json()
    #     print(league_data)
    #     print(queue_type)
        league = [elt for elt in league_data if elt['queueType'] == queue_type][0]
        wins = league['wins']
        losses = league['losses']
        winrate = wins / (wins + losses)
    #     if current_match_data['teams'][0]['teamId'] == latest_match_data['participants'][i]['teamId']:
        if i < num_players:
            team1.append(winrate)
        else:
            team2.append(winrate)
        print('%s: %s/%s (%.02f%%)' % (summoner_name, wins, wins + losses, winrate * 100) )
    except Exception as e:
        pass
        
print('----')
    
team1_sum = sum(team1)
team2_sum = sum(team2)
print('Predictions (Team 1 vs Team2):\n')
print('    Cumulative winrates: %.02f%% vs %.02f%%' % (team1_sum*100, team2_sum*100))

sum_ratio = team1_sum / team2_sum
print('    Win likelihood from sums: %.02f%% vs %.02f%%' % (1/(1+1/sum_ratio)*100, 1/(1+sum_ratio)*100))

# Sort for min
team1.sort(reverse = False)
team2.sort(reverse = False)
ratios = np.array(team1) / np.array(team2)
team1_ratio = np.mean(ratios) / np.mean(1/ratios)
print('    Win likelihood from mean min: %.02f%% vs %.02f%%' % (1/(1+1/team1_ratio)*100, 1/(1+team1_ratio)*100))

# Sort for max
team1.sort(reverse = False)
team2.sort(reverse = True)
ratios = np.array(team1) / np.array(team2)
team1_ratio = np.mean(ratios) / np.mean(1/ratios)
max_lh_1 = 1/(1+1/team1_ratio)*100
max_lh_2 = 1/(1+team1_ratio)*100
print('    Win likelihood from mean max: %.02f%% vs %.02f%%' % (max_lh_1, max_lh_2))

map_diff = 0.026
print('    After map side difference (%.02f%%): %.02f%% vs %.02f%%' % (map_diff, max_lh_1 * (1 - map_diff), max_lh_2 * (1 + map_diff)))

team1_carry = team1[-1]/team2[-1]
team2_carry = team2[0]/team1[0]
min_carry = min(team1_carry, team2_carry)
print('    Solo carry likelihood: %.02f vs %.02f' % (team1_carry/min_carry, team2_carry/min_carry))




Queue type: RANKED_SOLO_5x5
Queue id: 420
----
Team 1:
cj is awesome bu: 20/32 (62.50%)
ghost wingzero: 77/135 (57.04%)
WanderiN: 6/18 (33.33%)
UnknownMinx: 21/39 (53.85%)
SylowP: 27/59 (45.76%)

Team 2:
RedGhostzzz: 57/121 (47.11%)
ElReyMonito: 97/213 (45.54%)
Crysoooo: 101/212 (47.64%)
Uncle Fawka: 43/96 (44.79%)
----
Predictions (Team 1 vs Team2):

    Cumulative winrates: 252.48% vs 185.08%
    Win likelihood from sums: 57.70% vs 42.30%


ValueError: operands could not be broadcast together with shapes (5,) (4,) 

In [18]:
team1.sort(reverse = False)
team1

[0.46551724137931033,
 0.4864864864864865,
 0.48717948717948717,
 0.5189873417721519,
 0.5267857142857143]

In [19]:
team2.sort(reverse = True)
team2

[0.5353535353535354,
 0.5,
 0.46296296296296297,
 0.4090909090909091,
 0.2692307692307692]

In [20]:
ratios = np.array(team1) / np.array(team2)
print(ratios)
print(1/ratios)
print(np.mean(ratios))
print(np.mean(1/ratios))
team1_ratio = np.mean(ratios) / np.mean(1/ratios)
print(team1_ratio)
print(1/(1+1/team1_ratio))
print(1/(1+team1_ratio))
sum_ratio = team1_sum / team2_sum
print('--')
print(sum_ratio)
print(1/(1+1/sum_ratio))
print(1/(1+sum_ratio))

[0.86955107 0.97297297 1.05230769 1.26863572 1.95663265]
[1.15001871 1.02777778 0.9502924  0.78824834 0.51108214]
1.2240200232387322
0.8854838712484929
1.382317694293989
0.5802407032466109
0.419759296753389
--
1.1416487580591697
0.5330700254946419
0.4669299745053581


In [21]:
team1_w / team1_tot

0.4966442953020134

In [22]:
team2_w / team2_tot

0.4890254609306409

In [23]:
import numpy as np
np.mean(team1)

0.4969912542206301

In [17]:
team1_w, team2_w

(518, 557)

In [38]:
team1_tot, team2_tot

(539, 42)

In [2]:
current_match_data


{'gameId': 2830390741,
 'mapId': 11,
 'gameMode': 'CLASSIC',
 'gameType': 'MATCHED_GAME',
 'gameQueueConfigId': 420,
 'participants': [{'teamId': 100,
   'spell1Id': 11,
   'spell2Id': 4,
   'championId': 62,
   'profileIconId': 3541,
   'summonerName': 'cj is awesome bu',
   'bot': False,
   'summonerId': 45531592,
   'gameCustomizationObjects': [],
   'perks': {'perkIds': [8229, 8226, 8210, 8236, 8138, 8105],
    'perkStyle': 8200,
    'perkSubStyle': 8100}},
  {'teamId': 100,
   'spell1Id': 14,
   'spell2Id': 4,
   'championId': 157,
   'profileIconId': 3440,
   'summonerName': 'ghost wingzero',
   'bot': False,
   'summonerId': 43147057,
   'gameCustomizationObjects': [],
   'perks': {'perkIds': [8010, 9111, 9104, 8014, 8473, 8429],
    'perkStyle': 8000,
    'perkSubStyle': 8400}},
  {'teamId': 100,
   'spell1Id': 12,
   'spell2Id': 4,
   'championId': 41,
   'profileIconId': 3502,
   'summonerName': 'WanderiN',
   'bot': False,
   'summonerId': 33030069,
   'gameCustomizationObje